In [1]:
from psmiles import PolymerSmiles as PS
import pandas as pd
# Read csv and filter EGC
egc_df = pd.read_csv("/home/ibrahim/recources/export.csv")
egc_df = egc_df[egc_df.property == "Egc"]

#creating fingerprints for SS
smile_string_list = list(egc_df.smiles.values)
value_list = list(egc_df.value.values)
finger_print_list = []
for ss in smile_string_list:
    curren_polymere = PS(ss)
    
    
    finger_print_list.append(curren_polymere.fingerprint())

summary_list = list(zip(smile_string_list,finger_print_list,value_list))
fp_list_formatted = []
for i in finger_print_list:
    fp_list_formatted.append(i.tolist())
print(len(fp_list_formatted[0]))
ss_fp_egc_df = pd.DataFrame({'Smile': smile_string_list, 'Finger_prints': fp_list_formatted, 'Egc': value_list})
print(ss_fp_egc_df)

2048
                                                 Smile  \
0                                              [*]C[*]   
1                                          [*]CC([*])C   
2                                         [*]CC([*])CC   
3                                        [*]CC([*])CCC   
4                                     [*]CC([*])CC(C)C   
...                                                ...   
3375           [*]Nc1c([2H])c([2H])c([*])c([2H])c1[2H]   
3376               [*]CCCCCC[N+](C)(C)CCC[N+]([*])(C)C   
3377          [*]CCCCCCCC[N+](C)(C)CCCCCC[N+]([*])(C)C   
3378  [*]CCCCCCCCCCCCCCCC[N+](C)(C)CCCCCC[N+]([*])(C)C   
3379       [*]C=Cc1cc([Si](C)(C)C)c([*])cc1[Si](C)(C)C   

                                          Finger_prints     Egc  
0     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  6.8972  
1     [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  6.5196  
2     [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  6.5170  
3     [0, 1, 0, 0, 0, 0, 0, 0, 0, 

In [2]:
import numpy as np
import pandas as pd

final_df = pd.DataFrame(
    np.array(finger_print_list), index=smile_string_list
)

ss = pd.Series(value_list, index=smile_string_list, name="Egc")

concatenated_df = pd.concat(
    [final_df, ss],
    axis=1,
).reset_index(names="psmiles")
print(concatenated_df.set_index("psmiles").columns)

Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,     9,
       ...
        2039,  2040,  2041,  2042,  2043,  2044,  2045,  2046,  2047, 'Egc'],
      dtype='object', length=2049)


In [3]:
import pandas as pd



auto_ml_df = concatenated_df.sample(frac=1, random_state=0)#shulles the df
print(auto_ml_df.shape)


psmlies_column = auto_ml_df["psmiles"]
auto_ml_df = auto_ml_df.drop(columns=["psmiles"])
auto_ml_df.reset_index(drop=True, inplace=True)
print(auto_ml_df.shape)
split_index = int(len(auto_ml_df) * 0.8)

train_df = auto_ml_df[:split_index]
test_df = auto_ml_df[split_index:]
# evtl min max scaler 
train_df.columns = train_df.columns.astype(str)
test_df.columns = test_df.columns.astype(str)

test_df.head()

(3380, 2050)
(3380, 2049)


,0,1,2,3,4,5,6,7,8,9,...,2039,2040,2041,2042,2043,2044,2045,2046,2047,Egc
2704,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5.0497
2705,0,0,0,0,0,0,0,0,0,4,...,0,0,0,0,0,0,0,0,0,5.6991
2706,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.9694
2707,0,0,0,0,0,0,0,0,0,4,...,0,0,0,0,0,0,0,0,0,5.7012
2708,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5.3739


In [4]:
X_train = train_df.to_numpy()[:, :-1]
y_train = train_df.to_numpy()[:, -1]

X_test = test_df.to_numpy()[:, :-1] 
y_test = test_df.to_numpy()[:, -1]

(X_train.shape, y_train.shape), (X_test.shape, y_test.shape)

(((2704, 2048), (2704,)), ((676, 2048), (676,)))

In [5]:
%pip install gputil



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [23]:
import logging
import pprint


from ludwig.automl import auto_train



auto_train_results = auto_train(
    dataset=train_df,
    target='Egc',
    time_limit_s=14400,
    tune_for_memory=True
)

pprint.pprint(auto_train_results)


Analyzing fields: 100%|██████████| 2049/2049 [00:00<00:00, 2068.31it/s]
/home/ibrahim/.venv/lib/python3.10/site-packages/ludwig/schema/model_types/utils.py:238: UserWarning: Can't utilize `early_stop` while using a hyperopt scheduler. Setting early stop to -1.
  warnings.warn("Can't utilize `early_stop` while using a hyperopt scheduler. Setting early stop to -1.")
/home/ibrahim/.venv/lib/python3.10/site-packages/ludwig/hyperopt/utils.py:204: RuntimeWarning: All hyperopt parameters in Ludwig config are using grid_search space, but number of samples (10) is greater than 1. This will result in 9 duplicate trials being created. Consider setting `num_samples` to 1 in the hyperopt executor to prevent trial duplication.
  warnings.warn(


(pid=142287) Created a temporary directory at /tmp/tmpkkbqd8qs
(pid=142287) Writing /tmp/tmpkkbqd8qs/_remote_module_non_scriptable.py


(pid=142287) 2023-11-07 15:55:10.882449: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
(pid=142287) To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
(pid=142287) 2023-11-07 15:55:11.535116: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
(run_experiment_trial pid=142287) 2023-11-07 15:55:16,299	INFO util.py:549 -- Waiting for GPU util to reach 0.01. Util: 0.327


(pid=144411) Created a temporary directory at /tmp/tmp6b_j1qet
(pid=144411) Writing /tmp/tmp6b_j1qet/_remote_module_non_scriptable.py


(pid=144411) 2023-11-07 15:55:20.289201: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
(pid=144411) To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
(pid=144411) 2023-11-07 15:55:20.901828: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
(run_experiment_trial pid=142287) 2023-11-07 15:55:21,391	INFO util.py:549 -- Waiting for GPU util to reach 0.01. Util: 0.327
(run_experiment_trial pid=144411) 2023-11-07 15:55:25,546	INFO util.py:549 -- Waiting for GPU util to reach 0.01. Util: 0.168
(run_experiment_trial pid=142287) 2023-11-07 15:55:26,491	INFO util.py:549 -- Waiting for GPU util to reach 0.01. Util: 0.327


...

(pid=170635) 2023-11-07 15:57:33.130018: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
(pid=170635) To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
(run_experiment_trial pid=168511) 2023-11-07 15:57:33,380	INFO util.py:549 -- Waiting for GPU util to reach 0.01. Util: 0.168
(pid=170635) 2023-11-07 15:57:33.808113: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
(run_experiment_trial pid=166268) 2023-11-07 15:57:33,905	INFO util.py:549 -- Waiting for GPU util to reach 0.01. Util: 0.168
(run_experiment_trial pid=163950) 2023-11-07 15:57:33,990	INFO util.py:549 -- Waiting for GPU util to reach 0.01. Util: 0.327
(run_experiment_trial pid=168511) 2023-11-07 15:57:38,552	INFO util.py:549 -- Waiting for GPU util to reach 0.01. Util: 0.168
(run_experiment_trial

(pid=185932) Created a temporary directory at /tmp/tmpbpjzi07t
(pid=185932) Writing /tmp/tmpbpjzi07t/_remote_module_non_scriptable.py


(pid=185932) 2023-11-07 15:58:55.347542: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
(pid=185932) To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
(run_experiment_trial pid=166268) 2023-11-07 15:58:55,597	INFO util.py:549 -- Waiting for GPU util to reach 0.01. Util: 0.168
(run_experiment_trial pid=168511) 2023-11-07 15:58:55,588	INFO util.py:549 -- Waiting for GPU util to reach 0.01. Util: 0.168
(run_experiment_trial pid=170635) 2023-11-07 15:58:55,598	INFO util.py:549 -- Waiting for GPU util to reach 0.01. Util: 0.168
(pid=185932) 2023-11-07 15:58:56.014894: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
(run_experiment_trial pid=168511) 2023-11-07 15:59:00,696	INFO util.py:549 -- Waiting for GPU util to reach 0.01. Util: 0.168
(run_experiment_trial

(pid=188366) Created a temporary directory at /tmp/tmpq_5lgkxj
(pid=188366) Writing /tmp/tmpq_5lgkxj/_remote_module_non_scriptable.py


(run_experiment_trial pid=168511) 2023-11-07 15:59:05,773	INFO util.py:549 -- Waiting for GPU util to reach 0.01. Util: 0.168
(run_experiment_trial pid=170635) 2023-11-07 15:59:05,777	INFO util.py:549 -- Waiting for GPU util to reach 0.01. Util: 0.168
(run_experiment_trial pid=185932) 2023-11-07 15:59:05,834	INFO util.py:549 -- Waiting for GPU util to reach 0.01. Util: 0.327
(pid=188366) 2023-11-07 15:59:06.067354: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
(pid=188366) To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
(pid=188366) 2023-11-07 15:59:06.672021: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
(run_experiment_trial pid=170635) 2023-11-07 15:59:11,063	INFO util.py:549 -- Waiting for GPU util to reach 0.01. Util: 0.168
(run_experiment_trial

In [21]:
pprint.pprint(auto_train_results)
#predictions = auto_train_results.predict(test_df)


In [22]:
auto_train_results.best_model
best_model = auto_train_results.best_model()

2023-11-07 15:52:08,890	WARNING experiment_analysis.py:621 -- Could not find best trial. Did you pass the correct `metric` parameter?


ValueError: No best trial found. Please check if you specified the correct default metric (metric_score) and mode (min).

In [12]:
predictions = best_model.predict(test_df)
print("predictions :     ",predictions)
print("y_test:    " ,y_test)

AttributeError: 'AutoTrainResults' object has no attribute 'predict'

In [ ]:
eval_stats, _, _ = best_model.evaluate(dataset=test_df)
print(eval_stats)


In [ ]:
eval_stats, _, _ = best_model.evaluate(dataset=train_df)
print(eval_stats)

In [ ]:
import datetime

print(datetime.datetime.now())

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error,r2_score

mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test,predictions)
print("mae: ",mae," mse: ", mse," r2: ", r2)